# Reddit Climate Change - Modeling & Evaluation
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [ ]:
#  Topic detection information material

# LDA
# https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
# http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

# BERTopic
# https://maartengr.github.io/BERTopic/index.html


## Preparing the Environment and Data

In [2]:
# Preparing environment
#%pip install bertopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import pandas as pd
import re

/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [3]:
# Load data
clean_comments = pd.read_csv("data/preprocessed_comments.csv")

/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_96121/3432851145.py:2: DtypeWarning: Columns (0,1,2,4,7,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_comments = pd.read_csv("data/preprocessed_comments.csv")


In [25]:
# Na rows are again created and should be removed
clean_comments = clean_comments.dropna(axis=0)

In [8]:
# Convert timestamp because import creates float variables
clean_comments["created_year"] = pd.to_datetime(clean_comments["created_date"]).dt.strftime('%Y')
clean_comments["created_month"] = pd.to_datetime(clean_comments["created_date"]).dt.strftime('%m')
clean_comments["created_day"] = pd.to_datetime(clean_comments["created_date"]).dt.strftime('%d')

In [42]:
# remove climate change as word
clean_comments["body_clean"] = clean_comments["body_clean"].apply(lambda x: re.sub("climate change", "", x, flags=re.IGNORECASE))

# Modeling Topic Clusters

In [43]:
# Create a subset of dataframe for every year
year_groups = clean_comments.groupby(clean_comments['created_year'])

year_dfs = {'comments_{}'.format(year): group for year, group in year_groups}

for year, group in year_groups:
    year_dfs[year] = group

In [44]:
# Create separate text documents per year
docs_2010 = year_dfs["comments_2010"]["body_clean"].values
docs_2011 = year_dfs["comments_2011"]["body_clean"].values
docs_2012 = year_dfs["comments_2012"]["body_clean"].values
docs_2013 = year_dfs["comments_2013"]["body_clean"].values
docs_2014 = year_dfs["comments_2014"]["body_clean"].values
docs_2015 = year_dfs["comments_2015"]["body_clean"].values
docs_2016 = year_dfs["comments_2016"]["body_clean"].values
docs_2017 = year_dfs["comments_2017"]["body_clean"].values
docs_2018 = year_dfs["comments_2018"]["body_clean"].values
docs_2019 = year_dfs["comments_2019"]["body_clean"].values
docs_2020 = year_dfs["comments_2020"]["body_clean"].values
docs_2021 = year_dfs["comments_2021"]["body_clean"].values
docs_2022 = year_dfs["comments_2022"]["body_clean"].values

In [53]:
# Create variables to train model separately per year
topic_models = {}
doc_tuples = [(2010, docs_2010), (2011, docs_2011), (2012, docs_2012), (2013, docs_2013), (2014, docs_2014), (2015, docs_2015), (2016, docs_2016), (2017, docs_2017), (2018, docs_2018), (2019, docs_2019), (2020, docs_2020), (2021, docs_2021), (2022, docs_2022)]

In [54]:
# Train model
for year, year_docs in doc_tuples:

  # BERT stepwise
  # Step 1 - Extract embeddings
  embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

  # Step 2 - Reduce dimensionality
  umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

  # Step 3 - Cluster reduced embeddings
  hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

  # Step 4 - Tokenize topics
  vectorizer_model = CountVectorizer(stop_words="english")

  # Step 5 - Create topic representation
  ctfidf_model = ClassTfidfTransformer()

  # All steps together
  topic_model = BERTopic(
    embedding_model=embedding_model,    # Step 1 - Extract embeddings
    umap_model=umap_model,              # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,        # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
    calculate_probabilities=False,      # Raises speed
    min_topic_size = 300,               # Reduces number of topics
    nr_topics="auto",                    # Reduces number of topics
    verbose=True
  )

  topics, probs = topic_model.fit_transform(year_docs)
  topic_models[f"model_{year}"] = topic_model

Batches:   0%|          | 0/500 [00:00<?, ?it/s]

2023-05-08 13:25:33,175 - BERTopic - Transformed documents to Embeddings
2023-05-08 13:25:37,715 - BERTopic - Reduced dimensionality
2023-05-08 13:25:38,136 - BERTopic - Clustered reduced embeddings
2023-05-08 13:25:41,987 - BERTopic - Reduced number of topics from 103 to 16


Batches:   0%|          | 0/826 [00:00<?, ?it/s]

2023-05-08 13:33:01,008 - BERTopic - Transformed documents to Embeddings
2023-05-08 13:33:07,232 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

...

2023-05-08 20:14:19,076 - BERTopic - Clustered reduced embeddings
2023-05-08 20:14:37,541 - BERTopic - Reduced number of topics from 428 to 64


Batches:   0%|          | 0/3098 [00:00<?, ?it/s]

2023-05-08 21:18:19,875 - BERTopic - Transformed documents to Embeddings
2023-05-08 21:18:45,430 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-05-08 21:18:49,101 - BERTopic - Clustered reduced embeddings
2023-05-08 21:19:07,546 - BERTopic - Reduced number of topics from 430 to 104


In [87]:
# Check if model training was succesful
topic_models["model_2022"].get_topic_info()

,Topic,Count,Name
0,-1,49437,-1_people_just_like_climate
1,0,38326,0_people_like_just_don
2,1,1190,1_protest_protests_protesting_protesters
3,2,704,2_god_church_religion_bible
4,3,593,3_plastic_recycling_plastics_straws
5,4,577,4_trans_gender_women_men
6,5,464,5_fires_wildfires_forest_wildfire
7,6,419,6_gun_guns_shootings_mass
8,7,397,7_boomers_generation_gen_millennials
9,8,357,8_crypto_bitcoin_energy_mining


### Saving Model as CSV

In [72]:
# Save model as CSV file
for year in range(2010, 2023):
    model_name = f"model_{year}"
    docs_name = f"docs_{year}"
    file_name = f"data/model{year}.csv"
    topic_models[model_name].get_document_info(globals()[docs_name]).to_csv(file_name)

### Save Topic information as CSV file

In [166]:
# Save model as CSV file
for year in range(2010, 2023):
    model_name = f"model_{year}"
    file_name = f"data/topics_{year}.csv"
    topic_models[model_name].get_topic_info().to_csv(file_name)

### Combine Dataset with Detected Topics

In [151]:
for year in range(2010, 2023):
    # Create dataframes to be joined and drop irrelevant columns from topic dataframe, set indexes for join
    topic_data = topic_models[f"model_{year}"].get_document_info(globals()[f"docs_{year}"])
    topic_data = topic_data.drop(["Document","Probability","Representative_document"], axis=1).reset_index()

    original_data = year_dfs[f"comments_{year}"].reset_index(drop=True).reset_index()

    # Join Columns
    joined_df = original_data.join(topic_data,on='index',how='left',rsuffix='_topics')

    # Set the name of the joined dataframe
    joined_df_name = f"joined_{year}"

    # Add the joined dataframe to the global namespace
    globals()[joined_df_name] = joined_df


In [158]:
# Putting all dataframes back together
# List of joined dataframe names
joined_df_names = [f"joined_{year}" for year in range(2010, 2023)]

# List of joined dataframes
joined_dfs = [globals()[df_name] for df_name in joined_df_names]

# Concatenate the joined dataframes row-wise
concatenated_df = pd.concat(joined_dfs, axis=0, ignore_index=True)

### Saving Document with Topic Information

In [160]:
concatenated_df.to_csv("data/comments_with_topics.csv")

### Load Topic File & Insights

In [161]:
test = pd.read_csv("data/with_topics.csv")

In [163]:
test.isna().sum().sum()

0

In [164]:
# Sort out all outlier topics
test = test[test["Topic"] != -1]

In [165]:
# try to understand if topic detection worked
test = test.head(50)